# Generating C code for the right-hand sides of Maxwell's equations in Cartesian Coordinates

This tutorial will draw on previous work done by Ian Ruchlin on [Maxwell's equations in Cartesian Coordinates](Tutorial-MaxwellCurvilinear.ipynb), which itself drew on the two formulations described in [Illustrating Stability Properties of Numerical Relativity in Electrodynamics](https://arxiv.org/abs/gr-qc/0201051).This will be done to aid construction of an Einstein Toolkit thorn, which will itself be built in [the next tutorial](Tutorial-ETK_thorn-Maxwell.ipynb).

The construction of our equations here will be nearly identical; however, by assuming Cartesian coordinates, we are able to make several simplifications and eliminate the need for [reference_metric.py](../edit/reference_metric.py) as a dependency.

In [ ]:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *
